In [1]:
def used_items(agents, items, instance): 
    '''
    Takes in items list and returns list so that only items/bundles that are
    preferred by the agents are in the list.
    # Example
    n = ['A', 'B', 'C', 'D']
    items = [[1], [2], [3], [4]]
    instance3 = {
        "A": {"1": [1]}, 
        "B": {"1": [1], "2": [2]},
        "C": {"1": [1], "2": [2], "3": [3]},
        "D": {"1": [1], "2": [2,3], "3": [2,4], "4": [3,4]}
    }
    print(used_items(n, items, instance3))
    [[1], [2], [3], [2, 3], [2, 4], [3, 4]]
    '''
    items_check = items.copy() 
    for agent in agents:
        for item in items_check:
            if item in instance[agent].values():
                items_check.remove(item)
    for item in items_check:
        items.remove(item) # remove an item from items list if no agent has that item in preference list 
    for agent in instance:
        for pref in instance[agent]:
            if instance[agent][pref] not in items:
                items.append(instance[agent][pref]) # add in bundles to item list
    return items

In [2]:
def max_length(agents, items, instance):
    '''
    Finds the agent with the maximum length preference list
    and gives the value of the length of that list
    '''
    max_length = 0
    for agent_letter in instance:
        if len(instance[agent_letter]) > max_length:
            max_length = len(instance[agent_letter])
    return max_length

In [3]:
def same_length(agents, items, instance):
    '''
    Make all agents have preferences of same length.
    Do this by giving them empty lists.
    # Example
    n5 = ['A', 'B', 'C', 'D']
    instance5 = {
        "A": {"1": [1], "2": [2], "3": [3]}, 
        "B": {"1": [1], "2": [4]},
        "C": {"1": [2], "2": [5]},
        "D": {"1": [6]}
    }
    items5 = [[1], [2], [3], [4], [5], [6]]
    print(same_length(n5, items5, instance5))
    {'A': {'1': [1], '2': [2], '3': [3]},
     'B': {'1': [1], '2': [4], '3': []},
     'C': {'1': [2], '2': [5], '3': []},
     'D': {'1': [6], '2': [], '3': []}}
    '''
    lst = [] 
    for i in range(max_length(agents, items, instance) + 1): # Create list from 1 to max pref length 
        lst.append(i)
    lst.remove(0)
    for agent_letter in instance:
        for i in lst:
            if str(i) not in instance[agent_letter]:
                instance[agent_letter][str(i)] = [] # Give empty lists to make all preferences same length
    return instance

In [4]:
import itertools
def permutation(agents, items, instance):
    '''
    Get all permutations for the agents
    '''
    agent_permutation = []
    for i in itertools.permutations(agents):
        agent_permutation.append(list(i))
    return agent_permutation

In [5]:
def master_level_dict(agents, items, instance):
    '''
    Creates master dictionary to account for all the levels
    Each key in the dictionary is a level
    Each value corresponding to the key (level) is a dictionary of preferences going up to that level
    Example
    n5 = ['A', 'B', 'C', 'D']
    instance5 = {
        "A": {"1": [1], "2": [2], "3": [3]}, 
        "B": {"1": [1], "2": [4]},
        "C": {"1": [2], "2": [5]},
        "D": {"1": [6]}
    }
    items5 = [[1], [2], [3], [4], [5], [6]]
    master_level_dict(n5, items5, instance5)
    {'1': {'A': {'1': [1]}, 'B': {'1': [1]}, 'C': {'1': [2]}, 'D': {'1': [6]}},
     '2': {'A': {'1': [1], '2': [2]},
      'B': {'1': [1], '2': [4]},
      'C': {'1': [2], '2': [5]},
      'D': {'1': [6], '2': []}},
     '3': {'A': {'1': [1], '2': [2], '3': [3]},
      'B': {'1': [1], '2': [4], '3': []},
      'C': {'1': [2], '2': [5], '3': []},
      'D': {'1': [6], '2': [], '3': []}}}
    '''
    master_levels_list = {} # each element will be a list, with one more than level than the previous element
    lst = []
    levels = max_length(agents, items, instance)
    instance = same_length(n5, items5, instance5)
    for i in range(levels + 1): # Create list from 1 to max level
        lst.append(i)
    lst.remove(0)
    del(lst[-1]) # don't need to loop over final level, will add manually 
    count = 0
    while count < len(lst):
        l = []
        for i in range(count + 1):
            l.append(lst[i])
        master_levels_list[str(count + 1)] = l
        count += 1
    master_level_dict = {}
    for i in master_levels_list:
        x = master_levels_list[i]
        level_dict = {}
        for agt in agents:
            agt_dict = {}
            for i in x:
                agt_dict[str(i)] = instance[agt][str(i)]
            level_dict[agt] = agt_dict
        master_level_dict[str(i)] = level_dict
    master_level_dict[str(levels)] = instance # add final level, which is just original preference list
    return master_level_dict

In [6]:
def pr(agents, items, instance):
    '''
    Returns a dictionary for all allocations over every permutation for each level
    Example:
    n5 = ['A', 'B', 'C', 'D']
    instance5 = {
        "A": {"1": [1], "2": [2], "3": [3]}, 
        "B": {"1": [1], "2": [4]},
        "C": {"1": [2], "2": [5]},
        "D": {"1": [6]}
    }
    items5 = [[1], [2], [3], [4], [5], [6]]
    master_level_dict(n5, items5, instance5)
        {'1': {('A', 'B', 'C', 'D'): [[1], [], [2], [6]],
      ('A', 'B', 'D', 'C'): [[1], [], [6], [2]],
      ('A', 'C', 'B', 'D'): [[1], [2], [], [6]],
      ('A', 'C', 'D', 'B'): [[1], [2], [6], []],
      ('A', 'D', 'B', 'C'): [[1], [6], [], [2]],
      ('A', 'D', 'C', 'B'): [[1], [6], [2], []],
      ('B', 'A', 'C', 'D'): [[1], [], [2], [6]],
      ('B', 'A', 'D', 'C'): [[1], [], [6], [2]],
      ('B', 'C', 'A', 'D'): [[1], [2], [], [6]],
      ('B', 'C', 'D', 'A'): [[1], [2], [6], []],
      ('B', 'D', 'A', 'C'): [[1], [6], [], [2]],
      ('B', 'D', 'C', 'A'): [[1], [6], [2], []],
      ('C', 'A', 'B', 'D'): [[2], [1], [], [6]],
      ('C', 'A', 'D', 'B'): [[2], [1], [6], []],
      ('C', 'B', 'A', 'D'): [[2], [1], [], [6]],
      ('C', 'B', 'D', 'A'): [[2], [1], [6], []],
      ('C', 'D', 'A', 'B'): [[2], [6], [1], []],
      ('C', 'D', 'B', 'A'): [[2], [6], [1], []],
      ('D', 'A', 'B', 'C'): [[6], [1], [], [2]],
      ('D', 'A', 'C', 'B'): [[6], [1], [2], []],
      ('D', 'B', 'A', 'C'): [[6], [1], [], [2]],
      ('D', 'B', 'C', 'A'): [[6], [1], [2], []],
      ('D', 'C', 'A', 'B'): [[6], [2], [1], []],
      ('D', 'C', 'B', 'A'): [[6], [2], [1], []]},
     '2': {('A', 'B', 'C', 'D'): [[1], [4], [2], [6]],
      ('A', 'B', 'D', 'C'): [[1], [4], [6], [2]],
      ('A', 'C', 'B', 'D'): [[1], [2], [4], [6]],
      ('A', 'C', 'D', 'B'): [[1], [2], [6], [4]],
      ('A', 'D', 'B', 'C'): [[1], [6], [4], [2]],
      ('A', 'D', 'C', 'B'): [[1], [6], [2], [4]],
      ('B', 'A', 'C', 'D'): [[1], [2], [5], [6]],
      ('B', 'A', 'D', 'C'): [[1], [2], [6], [5]],
      ('B', 'C', 'A', 'D'): [[1], [2], [], [6]],
      ('B', 'C', 'D', 'A'): [[1], [2], [6], []],
      ('B', 'D', 'A', 'C'): [[1], [6], [2], [5]],
      ('B', 'D', 'C', 'A'): [[1], [6], [2], []],
      ('C', 'A', 'B', 'D'): [[2], [1], [4], [6]],
      ('C', 'A', 'D', 'B'): [[2], [1], [6], [4]],
      ('C', 'B', 'A', 'D'): [[2], [1], [], [6]],
      ('C', 'B', 'D', 'A'): [[2], [1], [6], []],
      ('C', 'D', 'A', 'B'): [[2], [6], [1], [4]],
      ('C', 'D', 'B', 'A'): [[2], [6], [1], []],
      ('D', 'A', 'B', 'C'): [[6], [1], [4], [2]],
      ('D', 'A', 'C', 'B'): [[6], [1], [2], [4]],
      ('D', 'B', 'A', 'C'): [[6], [1], [2], [5]],
      ('D', 'B', 'C', 'A'): [[6], [1], [2], []],
      ('D', 'C', 'A', 'B'): [[6], [2], [1], [4]],
      ('D', 'C', 'B', 'A'): [[6], [2], [1], []]},
     '3': {('A', 'B', 'C', 'D'): [[1], [4], [2], [6]],
      ('A', 'B', 'D', 'C'): [[1], [4], [6], [2]],
      ('A', 'C', 'B', 'D'): [[1], [2], [4], [6]],
      ('A', 'C', 'D', 'B'): [[1], [2], [6], [4]],
      ('A', 'D', 'B', 'C'): [[1], [6], [4], [2]],
      ('A', 'D', 'C', 'B'): [[1], [6], [2], [4]],
      ('B', 'A', 'C', 'D'): [[1], [2], [5], [6]],
      ('B', 'A', 'D', 'C'): [[1], [2], [6], [5]],
      ('B', 'C', 'A', 'D'): [[1], [2], [3], [6]],
      ('B', 'C', 'D', 'A'): [[1], [2], [6], [3]],
      ('B', 'D', 'A', 'C'): [[1], [6], [2], [5]],
      ('B', 'D', 'C', 'A'): [[1], [6], [2], [3]],
      ('C', 'A', 'B', 'D'): [[2], [1], [4], [6]],
      ('C', 'A', 'D', 'B'): [[2], [1], [6], [4]],
      ('C', 'B', 'A', 'D'): [[2], [1], [3], [6]],
      ('C', 'B', 'D', 'A'): [[2], [1], [6], [3]],
      ('C', 'D', 'A', 'B'): [[2], [6], [1], [4]],
      ('C', 'D', 'B', 'A'): [[2], [6], [1], [3]],
      ('D', 'A', 'B', 'C'): [[6], [1], [4], [2]],
      ('D', 'A', 'C', 'B'): [[6], [1], [2], [4]],
      ('D', 'B', 'A', 'C'): [[6], [1], [2], [5]],
      ('D', 'B', 'C', 'A'): [[6], [1], [2], [3]],
      ('D', 'C', 'A', 'B'): [[6], [2], [1], [4]],
      ('D', 'C', 'B', 'A'): [[6], [2], [1], [3]]}}
    '''
    items = used_items(agents, items, instance)
    levels = max_length(agents, items, instance)
    instance = same_length(agents, items, instance)
    agent_permutation = permutation(agents, items, instance)
    lst = [] # list of levels
    for i in range(levels + 1): # Create list from 1 to max level
        lst.append(i)
    lst.remove(0)  
    end_dict = {}
    for i in items: # Get rid of empty lists that may occur
        if i == []:
            items.remove([])
    for lvl in lst: # Loop over levels
        level_dict = master_level_dict(agents, items, instance)[str(lvl)]
        permutation_dict = {}
        for agt_perm in agent_permutation: # Loop over permutations
            perm_list = [] # allocation for each permutation
            items_copy = items.copy()
            for agt in agt_perm: # Loop over each individual agent
                for item in level_dict[str(agt)]:
                    count = 0 # flag to know when to add empty bundle 
                    if level_dict[str(agt)][item] in items_copy: # item or bundle has to be unallocated
                        perm_list.append(level_dict[agt][item])
                        items_copy.remove(level_dict[agt][item]) # remove item from unallocated list
                        for item2 in level_dict[agt][item]: # for items in allocated bundle
                            items_copy = list(filter(lambda r: r and (r.count(item2)==0),items_copy)) 
                            # if an item in a bundle is in another bundle, remove other bundle from unallocated list
                        count += 1
                        break # if agent has been allocated an item, go on to next agent
                if count == 0:
                    perm_list.append([]) # if no items to give, give empty bundle
            permutation_dict[tuple(agt_perm)] = perm_list
        end_dict[str(lvl)] = permutation_dict
    return end_dict

In [7]:
def mx(agents, items, instance):
    '''
    Returns an output allocation of the MX level (first tests for complete, then "highest" partial level)
    The output is all the maximal allocations of that level
    Example: 
    n5 = ['A', 'B', 'C', 'D']
    instance5 = {
        "A": {"1": [1], "2": [2], "3": [3]}, 
        "B": {"1": [1], "2": [4]},
        "C": {"1": [2], "2": [5]},
        "D": {"1": [6]}
    }
    items5 = [[1], [2], [3], [4], [5], [6]]
    mx(n5, items5, instance5)
    {('A', 'B', 'C', 'D'): [[1], [4], [2], [6]], ('A', 'B', 'D', 'C'): [[1], [4], [6], [2]], 
    ('A', 'C', 'B', 'D'): [[1], [2], [4], [6]], ('A', 'C', 'D', 'B'): [[1], [2], [6], [4]], 
    ('A', 'D', 'B', 'C'): [[1], [6], [4], [2]], ('A', 'D', 'C', 'B'): [[1], [6], [2], [4]], 
    ('B', 'A', 'C', 'D'): [[1], [2], [5], [6]], ('B', 'A', 'D', 'C'): [[1], [2], [6], [5]], 
    ('B', 'D', 'A', 'C'): [[1], [6], [2], [5]], ('C', 'A', 'B', 'D'): [[2], [1], [4], [6]], 
    ('C', 'A', 'D', 'B'): [[2], [1], [6], [4]], ('C', 'D', 'A', 'B'): [[2], [6], [1], [4]], 
    ('D', 'A', 'B', 'C'): [[6], [1], [4], [2]], ('D', 'A', 'C', 'B'): [[6], [1], [2], [4]], 
    ('D', 'B', 'A', 'C'): [[6], [1], [2], [5]], ('D', 'C', 'A', 'B'): [[6], [2], [1], [4]]}
    '''
    max_len = len(agents)
    end_dict = pr(agents, items, instance)
    flag = False # make sure to break when max level if first achieved 
    for i in end_dict:
        if flag:
            break
        for j in end_dict[i]:
            if flag:
                break
            count = 0
            for k in end_dict[i][j]:
                if k != []:
                    count += 1
                if count == max_len:
                    flag = True
                    complete_allocation_level = i # break at first sign of a complete allocation
                    break
    if flag:
        use_dict = end_dict[complete_allocation_level]
        complete_list = {}
        for q in use_dict:
            count = 0
            for k in use_dict[q]:
                if k != []:
                    count += 1
                if count == max_len:
                    complete_list[q] = use_dict[q]
    else:
        mx = {}
        max_allocation = {}
        for i in end_dict: # i is the level 1, 2, 3
            max_allocation_in_level = []
            for j in end_dict[i]: # j is ('A', 'B', 'C', 'D')
                count = 0
                for k in end_dict[i][j]:
                    if k != []:
                        count += 1
                    max_allocation_in_level.append(count)
            max_allocation[i] = max(max_allocation_in_level)
        max_partial = 0
        max_partial_level = 0
        for i in max_allocation:
            if max_allocation[i] > max_partial:
                max_partial = max_allocation[i]
                max_partial_level_amt = max_allocation[i]
                max_partial_level = i # 2
        complete_list = {}
        use_dict = end_dict[max_partial_level]
        for q in use_dict:
            count = 0
            for k in use_dict[q]:
                if k != []:
                    count += 1
                if count == max_partial_level_amt:
                    complete_list[q] = use_dict[q]
    return complete_list

In [8]:
import random
def output_allocation(agents, items, instance):
    '''
    Gives final, randomized maximal allocation.
    Example:
    n5 = ['A', 'B', 'C', 'D']
    instance5 = {
        "A": {"1": [1], "2": [2], "3": [3]}, 
        "B": {"1": [1], "2": [4]},
        "C": {"1": [2], "2": [5]},
        "D": {"1": [6]}
    }
    items5 = [[1], [2], [3], [4], [5], [6]]
    print(output_allocation(n5, items5, instance5))
    (('A', 'D', 'B', 'C'), 'with', [[1], [6], [4], [2]])
    '''
    complete_list = mx(agents, items, instance)
    length = len(complete_list) - 1
    x = random.randint(0, length)
    first_key = list(complete_list)[x]
    return (first_key, 'with', complete_list[first_key])

In [10]:
# Example 5
n5 = ['A', 'B', 'C', 'D']
instance5 = {
    "A": {"1": [1], "2": [2], "3": [3]}, 
    "B": {"1": [1], "2": [4]},
    "C": {"1": [2], "2": [5]},
    "D": {"1": [6]}
}
items5 = [[1], [2], [3], [4], [5], [6]]
print(output_allocation(n5, items5, instance5))

(('B', 'A', 'D', 'C'), 'with', [[1], [2], [6], [5]])


In [11]:
# Example 5
n5 = ['A', 'B', 'C', 'D']
instance5 = {
    "A": {"1": [1], "2": [2], "3": [3]}, 
    "B": {"1": [1], "2": [4]},
    "C": {"1": [2], "2": [5]},
    "D": {"1": [6]}
}
items5 = [[1], [2], [3], [4], [5], [6]]
print(output_allocation(n5, items5, instance5))

(('A', 'D', 'C', 'B'), 'with', [[1], [6], [2], [4]])
